In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Conectando o Google Drive ao Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Configurando PySpark no Google Colab

##### Instalação das dependências

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

##### Configurando as variáveis do ambiente

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# tornar o pyspark "importável"
import findspark
findspark.init('spark-2.4.4-bin-hadoop2.7')

##### Importando e Criando SparkSession

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("monkeypoxcolab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
spark

##### Instalando bibliotecas

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from IPython.core.display import HTML

### Carregando o dataset localizado no Google Drive

In [8]:
monkeypoxdf = spark.read.csv("/content/drive/MyDrive/Datasets/latest.csv", sep=",", header=True, inferSchema=True)

##### Mostrando os detalhes das colunas do dataframe

In [9]:
monkeypoxdf.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Country_ISO3: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Date_onset: timestamp (nullable = true)
 |-- Date_confirmation: timestamp (nullable = true)
 |-- Symptoms: string (nullable = true)
 |-- Hospitalised (Y/N/NA): string (nullable = true)
 |-- Date_hospitalisation: timestamp (nullable = true)
 |-- Isolated (Y/N/NA): string (nullable = true)
 |-- Date_isolation: timestamp (nullable = true)
 |-- Outcome: string (nullable = true)
 |-- Contact_comment: string (nullable = true)
 |-- Contact_ID: integer (nullable = true)
 |-- Contact_location: string (nullable = true)
 |-- Travel_history (Y/N/NA): string (nullable = true)
 |-- Travel_history_entry: string (nullable = true)
 |-- Travel_history_start: string (nullable = true)
 |-- Travel_

##### Renomeando as colunas

In [58]:
newmonkeypoxdf = monkeypoxdf.withColumnRenamed("Location", "Localizacao") \
              .withColumnRenamed("City", "Cidade") \
              .withColumnRenamed("Country", "Pais") \
              .withColumnRenamed("Country_ISO3", "Cod_ISO3") \
              .withColumnRenamed("Age", "Idade") \
              .withColumnRenamed("Gender", "Sexo") \
              .withColumnRenamed("Date_onset", "Data_entrada_sistema") \
              .withColumnRenamed("Date_confirmation", "Data_confirmacao") \
              .withColumnRenamed("Symptoms", "Sintomas") \
              .withColumnRenamed("Hospitalised (Y/N/NA)", "Hospitalizado") \
              .withColumnRenamed("Date_hospitalisation", "Data_internacao") \
              .withColumnRenamed("Isolated (Y/N/NA)", "Em_isolamento") \
              .withColumnRenamed("Date_isolation", "Data_isolamento") \
              .withColumnRenamed("Outcome", "Detectado") \
              .withColumnRenamed("Contact_comment", "Comentarios_contato") \
              .withColumnRenamed("Contact_ID", "Identidade_Contato") \
              .withColumnRenamed("Contact_location", "Cidade_contato") \
              .withColumnRenamed("Travel_history (Y/N/NA)", "Viajou") \
              .withColumnRenamed("Travel_history_entry", "Data_viagem") \
              .withColumnRenamed("Travel_history_start", "Data_inicio_viagem") \
              .withColumnRenamed("Travel_history_location", "Localidade_visitada") \
              .withColumnRenamed("Travel_history_country", "País_visitado") \
              .withColumnRenamed("Genomics_Metadata", "Genoma_virus") \
              .withColumnRenamed("Confirmation_method", "Metodo_confirmacao") \
              .withColumnRenamed("Source", "Fonte") \
              .withColumnRenamed("Source_II", "Fonte_II") \
              .withColumnRenamed("Source_III", "Fonte_III") \
              .withColumnRenamed("Source_IV", "Fonte_IV") \
              .withColumnRenamed("Source_V", "Fonte_V") \
              .withColumnRenamed("Source_VI", "Fonte_VI") \
              .withColumnRenamed("Source_VII", "Fonte_VII") \
              .withColumnRenamed("Date_entry", "Data_registro") \
              .withColumnRenamed("Date_death", "Data_obito") \
              .withColumnRenamed("Date_last_modified", "Data_ultima_atualizacao")
newmonkeypoxdf.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Localizacao: string (nullable = true)
 |-- Cidade: string (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Cod_ISO3: string (nullable = true)
 |-- Idade: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Data_entrada_sistema: timestamp (nullable = true)
 |-- Data_confirmacao: timestamp (nullable = true)
 |-- Sintomas: string (nullable = true)
 |-- Hospitalizado: string (nullable = true)
 |-- Data_internacao: timestamp (nullable = true)
 |-- Em_isolamento: string (nullable = true)
 |-- Data_isolamento: timestamp (nullable = true)
 |-- Detectado: string (nullable = true)
 |-- Comentarios_contato: string (nullable = true)
 |-- Identidade_Contato: integer (nullable = true)
 |-- Cidade_contato: string (nullable = true)
 |-- Viajou: string (nullable = true)
 |-- Data_viagem: string (nullable = true)
 |-- Data_inicio_viagem: string (nullable = true)
 |-- Localidade_visitada: string (nul

In [56]:
newmonkeypoxdf.show()

+---+---------+--------------------+---------+--------+--------+-----+----+--------------------+-------------------+--------------------+-------------+-------------------+-------------+-------------------+---------+--------------------+------------------+--------------+------+-----------+------------------+--------------------+-------------+------------------+------------------+--------------------+--------------------+---------+--------+-------+--------+---------+-------------------+----------+-----------------------+
| ID|   Status|         Localizacao|   Cidade|    País|Cod_ISO3|Idade|Sexo|Data_entrada_sistema|   Data_confirmacao|            Sintomas|Hospitalizado|    Data_internacao|Em_isolamento|    Data_isolamento|Detectado| Comentarios_contato|Identidade_Contato|Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem| Localidade_visitada|País_visitado|      Genoma_vírus|Metodo_confirmacao|               Fonte|            Fonte_II|Fonte_III|Fonte_IV|Fonte_V|Fonte_VI|Fonte_VII|      

In [59]:
newmonkeypoxdf.createOrReplaceTempView("MonkeyPox")
spark.sql("select ID, Status, Cidade, Pais, Idade, Sexo, Data_confirmacao, Sintomas from MonkeyPox ORDER BY Data_confirmacao asc").show(truncate=True)

+----+---------+-------------+-------+-----+------+----------------+--------------------+
|  ID|   Status|       Cidade|   Pais|Idade|  Sexo|Data_confirmacao|            Sintomas|
+----+---------+-------------+-------+-----+------+----------------+--------------------+
|N111|discarded|Costa del Sol|  Spain| null|  null|            null|                null|
|N321|discarded|         null| Israel| null|  null|            null|                null|
|N138|discarded|         null|  Spain| null|  null|            null|                null|
|N254|discarded|         null|  Spain| null|  null|            null|                null|
|N255|discarded|         null|Morocco| null|  null|            null|                null|
|N182|discarded|         null|  Spain| null|  null|            null|                null|
|N183|discarded|         null|  Spain| null|  null|            null|                null|
|N184|discarded|         null|  Spain| null|  null|            null|                null|
|N187|disc

#### Número de linhas no Dataframe 
##### *número total de casos rastreados no mundo incluindo a universalidade de status (confirmado, descartado, investigando e omit_error)*




In [60]:
newmonkeypoxdf.count()

49289

In [61]:
newmonkeypoxdf.select("Status").distinct().show()

+----------+
|    Status|
+----------+
|omit_error|
|      null|
| suspected|
| confirmed|
| discarded|
+----------+



#### Descrevendo aspectos das colunas

In [62]:
newmonkeypoxdf.describe().show()

+-------+--------------------+---------+-------------+-------------+-------+--------+-----+------+--------------------+-------------+-------------+---------+--------------------+------------------+--------------------+------+-----------+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+--------+---------+
|summary|                  ID|   Status|  Localizacao|       Cidade|   Pais|Cod_ISO3|Idade|  Sexo|            Sintomas|Hospitalizado|Em_isolamento|Detectado| Comentarios_contato|Identidade_Contato|      Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem| Localidade_visitada|       País_visitado|        Genoma_virus|  Metodo_confirmacao|               Fonte|            Fonte_II|           Fonte_III|            Fonte_IV|Fonte_V|Fonte_VI|Fonte_VII|
+-------+--------------------+---------+-------------+-------------+-------+--------+-----+-----

#### Ranking dos 10 países com maior número de infectados (até 22-08-2022)

In [64]:
newmonkeypoxdf_total = newmonkeypoxdf.groupBy("Pais").agg(count("Pais").alias("total_casos")).orderBy(col("total_casos").desc()).show(10)

+--------------------+-----------+
|                Pais|total_casos|
+--------------------+-----------+
|       United States|      16759|
|               Spain|       6470|
|              Brazil|       4186|
|             Germany|       3350|
|             England|       3191|
|              France|       2899|
|Democratic Republ...|       2380|
|              Canada|       1298|
|                Peru|       1210|
|         Netherlands|       1090|
+--------------------+-----------+
only showing top 10 rows



#### Sintomas encontrados entre os infectados pelo vírus

In [65]:
newmonkeypoxdf.select("Sintomas").distinct().count()

99

In [66]:
newmonkeypoxdf.select("Sintomas").distinct().show(99)

+--------------------+
|            Sintomas|
+--------------------+
|headache, skin le...|
|Perianal rash, fever|
|Fever, chills, fa...|
|headache, muscle ...|
|fever, outbreak o...|
|                rash|
|rash, body pains,...|
|             lesions|
|genital ulcer les...|
|characteristic sy...|
|      fever, lesions|
| skin manifestations|
|headache, muscle ...|
|symptoms compatib...|
|                Rash|
|      fever; myalgia|
|Spots on skin, ve...|
|         skin rashes|
|                null|
|  ulcerative lesions|
|fever, general di...|
|  myalgias, postules|
|muscle aches, fev...|
|blisters, high fever|
|Fatigue, headache...|
|skin lesions, ulc...|
|oral and genital ...|
|Malaise, headache...|
| fever, skin lesions|
|headache, fever, ...|
|            Vesicles|
|Pain urinating, f...|
|fatigue, fever, s...|
|skin lesions, hea...|
|fever, headache, ...|
|fever, cough, ski...|
|Perianal rash, fe...|
|         Rash, fever|
|skin lesions, fev...|
|fever, headache, ...|
|fatigue, s

In [67]:
#conversão de coluna do Dataframe para Pandas visando melhor visualização dos dados
newmonkeypoxdf.select("Sintomas").distinct().toPandas()

,Sintomas
0,"headache, skin lesions"
1,"Perianal rash, fever"
2,"Fever, chills, fatigue, headache, skin lesions"
3,"headache, muscle pain, back pain, vasicular ra..."
4,"fever, outbreak on the skin, hands, and chest"
...,...
94,"general weakness, fever, skin rashes"
95,genital ulcers
96,"Fever, skin rashes"
97,"Rash, muscle ache, fatigue"


In [69]:
newmonkeypoxdf.select("Sintomas","Sexo").distinct().toPandas()

,Sintomas,Sexo
0,lesions,male
1,"headache, skin lesions",None
2,ulcerative lesions,male
3,Rash,Male
4,"fever, skin lesions",male
...,...,...
106,"Swelling, fever, rash, diarrhea",Male
107,"lower abdomen skin lesions, fatigue, swollen l...",male
108,"Perianal rash, fever",Male
109,"rash, body pains, fever",male


### Panorama de infecção no Brasil

In [70]:
li=["Sao Paulo", "Rio de Janeiro", "Belo Horizonte"]
newmonkeypoxBrazil = newmonkeypoxdf.filter(newmonkeypoxdf.Cidade.isin(li)).show()

+-----+---------+------------+--------------+------+--------+-----+------+--------------------+-------------------+--------------+-------------+---------------+-------------+---------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+---------------+--------------------+------------------+--------------------+--------------------+--------------------+--------+-------+--------+---------+-------------------+----------+-----------------------+
|   ID|   Status| Localizacao|        Cidade|  Pais|Cod_ISO3|Idade|  Sexo|Data_entrada_sistema|   Data_confirmacao|      Sintomas|Hospitalizado|Data_internacao|Em_isolamento|Data_isolamento|Detectado|Comentarios_contato|Identidade_Contato|Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem|Localidade_visitada|  País_visitado|        Genoma_virus|Metodo_confirmacao|               Fonte|            Fonte_II|           Fonte_III|Fonte_IV|Fonte_V|Fonte_VI|Fonte_VII|      Data

In [71]:
newmonkeypoxdf.filter(newmonkeypoxdf.Sintomas.like("%skin%") & newmonkeypoxdf.Pais.like("%Brazil%")).show()

+-----+---------+--------------------+------------------+------+--------+-----+------+--------------------+-------------------+--------------------+-------------+-------------------+-------------+---------------+---------+-------------------+------------------+--------------+------+-----------+------------------+-------------------+---------------+------------+------------------+--------------------+--------------------+--------------------+--------------------+-------+--------+---------+-------------------+----------+-----------------------+
|   ID|   Status|         Localizacao|            Cidade|  Pais|Cod_ISO3|Idade|  Sexo|Data_entrada_sistema|   Data_confirmacao|            Sintomas|Hospitalizado|    Data_internacao|Em_isolamento|Data_isolamento|Detectado|Comentarios_contato|Identidade_Contato|Cidade_contato|Viajou|Data_viagem|Data_inicio_viagem|Localidade_visitada|  País_visitado|Genoma_virus|Metodo_confirmacao|               Fonte|            Fonte_II|           Fonte_III|      